In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [2]:
df = sc.textFile("gs://ds-url-catag/stick_bytype/all_stickers/*") \
    .map(lambda line: line.split(",")).map(lambda line: (line[0],line[1])).toDF(['id','type'])

In [41]:
df.count()

367791

In [3]:
from pyspark.sql.types import StringType
import re
remove_quotes = udf(lambda x: re.sub('"','',x), StringType())
remove_slashes = udf(lambda x: re.sub('\\\\','',x), StringType())

In [4]:
new_df = df.select(*[remove_quotes(column).alias(column) for column in df.columns])
new_df =  new_df.select(*[remove_slashes(column).alias(column) for column in df.columns])

In [44]:
new_df.dropDuplicates().count()

1695

In [45]:
new_df.groupby('type').agg(countDistinct('id').alias('cnt')).show()

+------------+----+
|        type| cnt|
+------------+----+
|        FREE| 385|
|SUBSCRIPTION| 113|
|        PAID|1047|
|DISCONTINUED| 150|
+------------+----+



In [5]:
df_send = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/transposed/BBM-STICKER-SEND/")

In [47]:
# from pyspark.sql.types import IntegerType
# from pyspark.sql.functions import udf

# size_ = udf(lambda xs: len(xs), IntegerType())

# df_send.select('Country').where(size_(col("Country")) <= 2).distinct().show(400)


In [6]:
data_join = df_send.join(new_df,new_df.id == df_send.sticker_pack_id).select([df_send.sticker_pack_id]+[new_df.id,new_df.type])

In [ ]:
data_join.select('sticker_pack_id').distinct().count()

In [ ]:
df_send.select('sticker_pack_id').distinct().count()

1699